In [1]:
%load_ext autoreload
%autoreload 2

## Optimization problem

In [63]:
from pbp_utils import get_starters_of_game
starters_A, starters_B, boxscore_A, boxscore_B = get_starters_of_game('https://www.fiba.basketball/en/events/fiba-olympic-qualifying-tournament-2024-piraeus-greece/games/121092-DOM-GRE')

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000103136a80 chromedriver + 4385408
1   chromedriver                        0x000000010312f38c chromedriver + 4354956
2   chromedriver                        0x0000000102d4cb0c chromedriver + 281356
3   chromedriver                        0x0000000102d8f2f8 chromedriver + 553720
4   chromedriver                        0x0000000102dc7d24 chromedriver + 785700
5   chromedriver                        0x0000000102d83eec chromedriver + 507628
6   chromedriver                        0x0000000102d848c4 chromedriver + 510148
7   chromedriver                        0x00000001030fe43c chromedriver + 4154428
8   chromedriver                        0x0000000103102ea0 chromedriver + 4173472
9   chromedriver                        0x00000001030e3ff8 chromedriver + 4046840
10  chromedriver                        0x000000010310378c chromedriver + 4175756
11  chromedriver                        0x00000001030d6fb8 chro

In [3]:
boxscore_A

,#,Players,Min,Pts,FG,2Pts,3Pts,FT,OREB,DREB,...,PF,TO,STL,BLK,+/-,EFF,FG%,FT%,2Pts%,3Pts%
0,4,Tyrese Haliburton,30:49,6,2/8,0/2,2/6,0/0,0,5,...,0,1,1,1,10,13,25%,-,0%,33.3%
1,5,Mikal Bridges,35:52,19,6/12,3/6,3/6,4/6,5,4,...,2,3,2,0,-14,23,50%,66.7%,50%,50%
2,6,Cameron Johnson,07:02,3,1/1,0/0,1/1,0/2,0,1,...,1,0,0,0,8,3,100%,0%,-,100%
5,9,Bobby Portis Jr,20:02,14,5/8,5/6,0/2,4/4,1,5,...,3,1,1,0,6,18,62.5%,100%,83.3%,0%
6,10,Anthony Edwards,37:00,24,9/20,8/16,1/4,5/8,0,5,...,2,2,1,1,-20,18,45%,62.5%,50%,25%
7,11,Jalen Brunson,22:37,13,5/11,5/9,0/2,3/3,0,1,...,3,1,0,0,-17,9,45.5%,100%,55.6%,0%
8,12,Josh Hart,22:00,10,4/4,3/3,1/1,1/2,0,1,...,5,2,0,0,-5,10,100%,50%,100%,100%
10,14,Walker Kessler,16:01,6,3/3,3/3,0/0,0/0,4,3,...,2,1,0,1,-1,13,100%,-,100%,-
11,15,Austin Reaves,33:37,23,7/14,5/9,2/5,7/7,2,3,...,3,3,1,0,-12,20,50%,100%,55.6%,40%
12,Team/Coaches,Team/Coaches,NaN,NaN,NaN,NaN,NaN,NaN,3,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from cpsat_utils import fixing_time_boxscore
boxscore_A_players = fixing_time_boxscore(boxscore_A)
boxscore_B_players = fixing_time_boxscore(boxscore_B)
boxscore_B_players.time_seconds.tolist()

[1930.0, 1072.0, 2413.0, 433.0, 1826.0, 2319.0, 682.0, 138.0, 153.0, 2534.0]

In [5]:
from bs4 import BeautifulSoup
import urllib.request as urllib
from pbp_utils import concatenating_both_teams_quarter, get_country_trigrams_dict
from cpsat_utils import custom_sort_pbp
import pandas as pd

country_trigrams = get_country_trigrams_dict()

url = 'https://www.fiba.basketball/basketballworldcup/2023/game/1009/USA-Canada'
soup = BeautifulSoup(urllib.urlopen(url), "html.parser")

print("Locating Play By Play...")

total_quarters_time = int(boxscore_A[boxscore_A['#']=='Totals']['Min'].values[0])
if total_quarters_time==200:
    total_quarters =  4
else:
    ot_time = int((total_quarters_time - 200)/ 25)
    total_quarters = 4 + ot_time

all_pbp = pd.DataFrame()
home_team = country_trigrams[url.split('/')[-1].split('-')[0]]
away_team = country_trigrams[url.split('/')[-1].split('-')[1]]
for i in range(1, total_quarters+1):
    pbpQ_ = concatenating_both_teams_quarter(total_quarters, i, soup, home_team, away_team)
    pbpQ= custom_sort_pbp(pbpQ_)
    all_pbp = pd.concat([all_pbp, pbpQ])



Locating Play By Play...


In [11]:
all_pbp['quarter'] = all_pbp['Period'].str[-1]
all_pbp.drop_duplicates(inplace=True)
all_pbp

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,7,Dwight Powell,jump ball won,2700.0,1,NaN
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,14,Walker Kessler,jump ball lost,2700.0,1,0.0
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:37,24,Dillon Brooks,2pt hook shot made,2677.0,1,23.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,14,Walker Kessler,offensive rebound,2655.0,1,22.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,10,Anthony Edwards,2pt driving layup missed,2655.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,CAN,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:20,24,Dillon Brooks,2nd of 2 free throws made,20.0,1,0.0
0,CAN,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:14,9,RJ Barrett,defensive rebound,14.0,1,6.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:14,15,Austin Reaves,3pt pullup jump shot missed,14.0,1,0.0
0,None,None,111,111,OT1,00:00,,,End of quarter,0.0,1,14.0


In [12]:
from cpsat_utils import remove_inconsistent_rows
all_pbp = remove_inconsistent_rows(all_pbp)

/Users/nildenver/Desktop/Canada Team/Canada-Team-Olympics/cpsat_utils.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_row = df.loc[:index-1][(df['Player'] == player)].tail(1)


In [13]:
# # all_pbp['Total_Seconds'] = all_pbp['Total_Seconds']+(4-all_pbp['quarter'].astype(int))*600
# for number in boxscore_A_players['#'].tolist():
#     column_name = f'X_{number}_j_A'
#     all_pbp[column_name] = 0
#     if number in boxscore_A[boxscore_A.Players.isin(starters_A)]['#'].tolist():
#         all_pbp[column_name].iloc[0] = 1    
# for number in boxscore_B_players['#'].tolist():
#     column_name = f'X_{number}_j_B'
#     all_pbp[column_name] = 0
#     if number in boxscore_B[boxscore_B.Players.isin(starters_B)]['#'].tolist():
#         all_pbp[column_name].iloc[0] = 1   
# all_pbp.reset_index(drop=True, inplace=True)
# all_pbp

In [14]:
# variables_cols = [col for col in all_pbp.columns if 'X_' in col]
# for i, (index, row) in enumerate(all_pbp.iterrows()):
#     if i > 0:
#         if 'ubstitution' not in row['Action_Description']:
#             all_pbp.loc[index, variables_cols] = all_pbp.loc[index - 1, variables_cols]
#         else:
#             all_pbp.loc[index, variables_cols] = all_pbp.loc[index - 1, variables_cols]
#             team = row['Team']
#             player_number = row['Number']
#             action_description = row['Action_Description']
            
#             if 'out' in action_description:
#                 value_to_set = 0
#             elif 'in' in action_description:
#                 value_to_set = 1
#             else:
#                 # Handle other cases, if needed
#                 value_to_set = None
                
#             if team == 'ANG':
#                 variable_name = f'X_{player_number}_j_A'
#             elif team == 'ITA':
#                 variable_name = f'X_{player_number}_j_B'
#             else:
#                 # Handle other team names, if needed
#                 variable_name = None
            
#             if variable_name is not None:
#                 all_pbp.loc[index, variable_name] = value_to_set

In [15]:
# all_pbp[[col for col in variables_cols if 'B' in col]]

In [16]:
all_pbp

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,7,Dwight Powell,jump ball won,2700.0,1,NaN
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,14,Walker Kessler,jump ball lost,2700.0,1,0.0
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:37,24,Dillon Brooks,2pt hook shot made,2677.0,1,23.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,14,Walker Kessler,offensive rebound,2655.0,1,22.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,10,Anthony Edwards,2pt driving layup missed,2655.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,CAN,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:20,24,Dillon Brooks,2nd of 2 free throws made,20.0,1,0.0
0,CAN,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:14,9,RJ Barrett,defensive rebound,14.0,1,6.0
0,USA,https://www.fiba.basketball/api/img/team/logof...,118,127,OT1,0:14,15,Austin Reaves,3pt pullup jump shot missed,14.0,1,0.0
0,None,None,111,111,OT1,00:00,,,End of quarter,0.0,1,14.0


In [17]:
# all_pbp = all_pbp[(~all_pbp.Number.isnull())&(all_pbp.Number!='')].reset_index(drop=True)

In [18]:
# all_pbp_team_A = all_pbp[(all_pbp.Team=='ANG')&(all_pbp.Action_Description!='Timeout')].reset_index(drop=True)
# all_pbp_team_B = all_pbp[(all_pbp.Team=='ITA')&(all_pbp.Action_Description!='Timeout')].reset_index(drop=True)
# all_pbp_team_A

In [19]:
# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp.index[all_pbp['Action_Description'] == 'Substitution in']
indices_out = all_pbp.index[all_pbp['Action_Description'] == 'Substitution out']
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        
        in_row = all_pbp.iloc[index_in].copy()
        out_row =all_pbp.iloc[index_out].copy()
        all_pbp.iloc[index_in] =  out_row
        all_pbp.iloc[index_out] = in_row
all_pbp.reset_index(drop=True, inplace=True)

'''################################################################################################ For only team B

# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution in']
indices_out = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution out']
print(indices_out)
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        print(index_in, index_out)
        in_row = all_pbp_team_B.iloc[index_in].copy()
        out_row =all_pbp_team_B.iloc[index_out].copy()
        all_pbp_team_B.iloc[index_in] =  out_row
        all_pbp_team_B.iloc[index_out] = in_row
all_pbp_team_B.reset_index(drop=True, inplace=True)

################################################################################################ For only team A

# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp_team_A.index[all_pbp_team_A['Action_Description'] == 'Substitution in']
indices_out = all_pbp_team_A.index[all_pbp_team_A['Action_Description'] == 'Substitution out']
print(indices_out)
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        print(index_in, index_out)
        in_row = all_pbp_team_A.iloc[index_in].copy()
        out_row =all_pbp_team_A.iloc[index_out].copy()
        all_pbp_team_A.iloc[index_in] =  out_row
        all_pbp_team_A.iloc[index_out] = in_row
all_pbp_team_A.reset_index(drop=True, inplace=True)

'''

"################################################################################################ For only team B\n\n# Finding indices of rows with 'Substitution in' and 'Substitution out'\nindices_in = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution in']\nindices_out = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution out']\nprint(indices_out)\nindices = tuple(zip(indices_in, indices_out))\n# Reordering rows if 'Substitution out' comes after 'Substitution in'\nfor index_in,index_out in indices:\n    if index_out > index_in:\n        # Swap the rows\n        print(index_in, index_out)\n        in_row = all_pbp_team_B.iloc[index_in].copy()\n        out_row =all_pbp_team_B.iloc[index_out].copy()\n        all_pbp_team_B.iloc[index_in] =  out_row\n        all_pbp_team_B.iloc[index_out] = in_row\nall_pbp_team_B.reset_index(drop=True, inplace=True)\n\n##########################################################################################

In [20]:
# import pandas as pd

# # Find the indices where "Action_Description" is "Substitution out" and "time_elapsed" is greater than 0
# all_pbp['last_time'] = all_pbp['Total_Seconds'].shift(1)
# indices = all_pbp[(all_pbp['Action_Description'] == 'Substitution out') & (all_pbp['last_time'] -all_pbp['Total_Seconds']> 0)].index

# # Create new rows with the same information
# new_rows = [{
#     'Period': row['Period'],
#     'Time': row['Time'],
#     'Number': '',
#     'Player': '',
#     'Action_Description': 'out of bounds',
#     'Total_Seconds': row['Total_Seconds'],
#     'last_time': row['last_time']
# } for _, row in all_pbp.iloc[indices].iterrows()]

# # Insert the new rows before the corresponding rows
# for idx, df in enumerate(new_rows):
#     all_pbp = pd.concat([all_pbp.iloc[:indices[idx]], pd.DataFrame([df]), all_pbp.iloc[indices[idx]:]], ignore_index=True)

# # all_pbp = pd.concat([all_pbp.iloc[:idx] for idx in indices] + [pd.DataFrame(new_rows)] + [all_pbp.iloc[idx:] for idx in indices], ignore_index=True)

# # # Reset the index
# all_pbp.reset_index(drop=True, inplace=True)


In [21]:
all_pbp['time_elapsed'] = all_pbp['Total_Seconds'].shift(1)- all_pbp['Total_Seconds']
# all_pbp_team_A['time_elapsed'] = all_pbp_team_A['Total_Seconds'].shift(1)- all_pbp_team_A['Total_Seconds']
# all_pbp_team_B['time_elapsed'] = all_pbp_team_B['Total_Seconds'].shift(1)- all_pbp_team_B['Total_Seconds']


In [22]:
all_pbp[583:]

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
583,USA,https://www.fiba.basketball/api/img/team/logof...,115,120,OT1,2:20,10,Anthony Edwards,2nd of 2 free throws missed,140.0,1,0.0
584,CAN,https://www.fiba.basketball/api/img/team/logof...,115,120,OT1,2:20,9,RJ Barrett,defensive rebound,140.0,1,0.0
585,USA,https://www.fiba.basketball/api/img/team/logof...,115,120,OT1,2:20,10,Anthony Edwards,1st of 2 free throws made,140.0,1,0.0
586,CAN,https://www.fiba.basketball/api/img/team/logof...,115,120,OT1,1:58,24,Dillon Brooks,foul drawn,118.0,1,22.0
587,USA,https://www.fiba.basketball/api/img/team/logof...,115,120,OT1,1:58,11,Jalen Brunson,personal foul; 2 free throws awarded,118.0,1,0.0
588,CAN,https://www.fiba.basketball/api/img/team/logof...,115,121,OT1,1:58,24,Dillon Brooks,1st of 2 free throws made,118.0,1,0.0
589,CAN,https://www.fiba.basketball/api/img/team/logof...,115,121,OT1,1:58,24,Dillon Brooks,2nd of 2 free throws missed,118.0,1,0.0
590,CAN,https://www.fiba.basketball/api/img/team/logof...,115,121,OT1,1:58,0,Luguentz Dort,offensive rebound,118.0,1,0.0
591,CAN,https://www.fiba.basketball/api/img/team/logof...,115,121,OT1,1:47,2,Shai Gilgeous-Alexander,2pt driving layup missed,107.0,1,11.0
592,USA,https://www.fiba.basketball/api/img/team/logof...,115,121,OT1,1:47,10,Anthony Edwards,defensive rebound,107.0,1,0.0


In [25]:
all_pbp[(all_pbp.time_elapsed!=0)].time_elapsed.sum()

2700.0

In [26]:
all_pbp[['Period','Time', 'Number', 'Player', 'Action_Description', 'Total_Seconds', 'time_elapsed']][550:]

,Period,Time,Number,Player,Action_Description,Total_Seconds,time_elapsed
550,Q4,0:00,0,Luguentz Dort,Substitution in,300.0,0.0
551,Q4,0:00,13,Kelly Olynyk,Substitution in,300.0,0.0
552,Q4,0:00,10,Anthony Edwards,Substitution in,300.0,0.0
553,OT1,5:00,None,,jump ball situation; throw-in,300.0,0.0
554,OT1,4:51,7,Dwight Powell,made the assist,291.0,9.0
...,...,...,...,...,...,...,...
618,OT1,0:20,24,Dillon Brooks,2nd of 2 free throws made,20.0,0.0
619,OT1,0:14,9,RJ Barrett,defensive rebound,14.0,6.0
620,OT1,0:14,15,Austin Reaves,3pt pullup jump shot missed,14.0,0.0
621,OT1,00:00,,,End of quarter,0.0,14.0


vale i el model seria el següent: 
- sum(x_ij for i in players) == 5,     valid for all j actions
- sum(x_ij * time_elapsed_ij == truth_total_time[i]),      valid for all i players
- x_ij == 1,            valid for all j actions and i players such that "Player" column corresponds to player i

In [27]:
boxscore_A

,#,Players,Min,Pts,FG,2Pts,3Pts,FT,OREB,DREB,...,TO,STL,BLK,+/-,EFF,FG%,FT%,2Pts%,3Pts%,time_seconds
0,4,Tyrese Haliburton,30:49,6,2/8,0/2,2/6,0/0,0,5,...,1,1,1,10,13,25%,-,0%,33.3%,1849.0
1,5,Mikal Bridges,35:52,19,6/12,3/6,3/6,4/6,5,4,...,3,2,0,-14,23,50%,66.7%,50%,50%,2152.0
2,6,Cameron Johnson,07:02,3,1/1,0/0,1/1,0/2,0,1,...,0,0,0,8,3,100%,0%,-,100%,422.0
5,9,Bobby Portis Jr,20:02,14,5/8,5/6,0/2,4/4,1,5,...,1,1,0,6,18,62.5%,100%,83.3%,0%,1202.0
6,10,Anthony Edwards,37:00,24,9/20,8/16,1/4,5/8,0,5,...,2,1,1,-20,18,45%,62.5%,50%,25%,2220.0
7,11,Jalen Brunson,22:37,13,5/11,5/9,0/2,3/3,0,1,...,1,0,0,-17,9,45.5%,100%,55.6%,0%,1357.0
8,12,Josh Hart,22:00,10,4/4,3/3,1/1,1/2,0,1,...,2,0,0,-5,10,100%,50%,100%,100%,1320.0
10,14,Walker Kessler,16:01,6,3/3,3/3,0/0,0/0,4,3,...,1,0,1,-1,13,100%,-,100%,-,961.0
11,15,Austin Reaves,33:37,23,7/14,5/9,2/5,7/7,2,3,...,3,1,0,-12,20,50%,100%,55.6%,40%,2017.0
12,Team/Coaches,Team/Coaches,NaN,NaN,NaN,NaN,NaN,NaN,3,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
all_pbp.head()

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,7,Dwight Powell,jump ball won,2700.0,1,NaN
1,USA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,14,Walker Kessler,jump ball lost,2700.0,1,0.0
2,CAN,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:37,24,Dillon Brooks,2pt hook shot made,2677.0,1,23.0
3,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,14,Walker Kessler,offensive rebound,2655.0,1,22.0
4,USA,https://www.fiba.basketball/api/img/team/logof...,0,2,Q1,9:15,10,Anthony Edwards,2pt driving layup missed,2655.0,1,0.0


In [29]:
starters_A

['Mikal Bridges',
 'Anthony Edwards',
 'Jalen Brunson',
 'Josh Hart',
 'Walker Kessler']

In [34]:
from cpsat_utils import solve_optimization_problem
from ortools.sat.python import cp_model

# Example usage
# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list
truth_total_time =[int(time) for time in boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'time_seconds'].tolist()]  # Assuming 'truth_total_time' is the column containing total time for each player
team_names =boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'Players'].tolist() 
time_elapsed = all_pbp['time_elapsed'].fillna(0).tolist()
players_actions = all_pbp['Player'].str.replace('  ', ' ').tolist()
descriptions = all_pbp['Action_Description'].tolist()
quarters = all_pbp['Period'].tolist()
team_action = all_pbp['Team'].tolist()


N = len(team_names)
T = len(all_pbp)
print(N, T)

solution_X_ij_A = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_A, quarters, team_action, home_team, total_quarters)
if solution_X_ij_A is not None:
    print("Solution for X_ij_A:")
    for i,row in enumerate(solution_X_ij_A):
        all_pbp.loc[:, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] = row
        print(row)
else:
    print("No solution found.")


9 623
CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 1
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.01485
usertime: 0.01485
deterministic_time: 1.692e-05
gap_integral: 0
solution_fingerprint: 0x4d709da83a36d4c

Solution for X_ij_A:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [35]:
i =0
sum_pred = 0
sum_actual = 0
for col in all_pbp.columns:
    if 'j_A' in col:
        print(f'For player number {col.split("_")[1]}: ----- We predict {sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))} vs {boxscore_A.time_seconds.iloc[i]} real seconds')
        sum_pred += sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))
        sum_actual += boxscore_A.time_seconds.iloc[i]
        i += 1


        print('-----------------------')
print('Total sum seconds predicted', sum_pred)
print('Total sum of real seconds', sum_actual)

For player number 4: ----- We predict 1849 vs 1849.0 real seconds
-----------------------
For player number 5: ----- We predict 2152 vs 2152.0 real seconds
-----------------------
For player number 6: ----- We predict 422 vs 422.0 real seconds
-----------------------
For player number 9: ----- We predict 1202 vs 1202.0 real seconds
-----------------------
For player number 10: ----- We predict 2220 vs 2220.0 real seconds
-----------------------
For player number 11: ----- We predict 1357 vs 1357.0 real seconds
-----------------------
For player number 12: ----- We predict 1320 vs 1320.0 real seconds
-----------------------
For player number 14: ----- We predict 961 vs 961.0 real seconds
-----------------------
For player number 15: ----- We predict 2017 vs 2017.0 real seconds
-----------------------
Total sum seconds predicted 13500
Total sum of real seconds 13500.0


In [61]:
def solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, player_action, descriptions, starters, quarters, team_action, team, total_quarters, flag_error=False):
    model = cp_model.CpModel()

    # Variables
    X_ij_A = [[model.NewIntVar(0, 1, f'X_{i}_{j}_A') for j in range(T)] for i in range(N)]

    # Constraints
    player_name_index_to_sub_out = []
    for j in range(T):
        if j==0:
            for i in range(N):
                if team_names[i] in starters:
                    model.Add(X_ij_A[i][j] == 1)

        model.Add(sum(X_ij_A[i][j] for i in range(N)) == 5)
        players_popped = 0
        for i in range(N):
            # Adding constraint only for actions corresponding to player i
            if 'ubstitution' not in descriptions[j] or team_action[j] != team:
                if quarters[j] == quarters[j-1]:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)
                if player_action[j] == team_names[i] and not flag_error:
                    model.Add(X_ij_A[i][j] == 1)

            elif 'ubstitution out' in descriptions[j] and team_action[j] == team:
                if player_action[j] == team_names[i]:
                    # If going out, the player was previously in the game
                    model.Add( X_ij_A[i][j-1] == 1)  
                    player_name_index_to_sub_out.append((i,j, team_names[i]))
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)


            elif 'ubstitution in' in descriptions[j] and team_action[j] == team:
                if len(player_name_index_to_sub_out) > 0 and players_popped == 0:
                    player_index, time_index, player_name = player_name_index_to_sub_out[0]
                # Check if the player is involved in the substitution
                if player_action[j] == team_names[i]:
                    # If going in, the player should transition from 0 to 1
                    model.Add(X_ij_A[i][j] == 1)
                elif i ==  player_index and players_popped == 0:
                    model.Add(X_ij_A[player_index][j] == 0)
                    player_name_index_to_sub_out.pop(0)  # Remove the player from the list    
                    players_popped += 1           
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)

                    
                

            # Total time constraint for player i 
    model.Add(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T) for i in range(N)) == (4*3000 +(total_quarters-4)*1500))




    objective_expr = [model.NewIntVar(-100000, 100000, f'Objective_{i}') for i in range(N)]
    for i in range(N):
        model.AddAbsEquality(objective_expr[i], sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - truth_total_time[i])

    # Objective
    
    # model.Minimize(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - X_ij_A_values.loc[j, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] for i in range(N) for j in range(T)))
    model.Minimize(sum(objective_expr))


    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print(solver.ResponseStats())

    if status == cp_model.OPTIMAL:
        # Retrieve solution
        solution_X_ij_A = [[solver.Value(X_ij_A[i][j]) for j in range(T)] for i in range(N)]
        return solution_X_ij_A
    else:
        return None
    
# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list
truth_total_time =[int(time) for time in boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'time_seconds'].tolist()]  # Assuming 'truth_total_time' is the column containing total time for each player
team_names =boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'Players'].tolist() 
time_elapsed = all_pbp['time_elapsed'].fillna(0).tolist()
players_actions = all_pbp['Player'].tolist()
descriptions = all_pbp['Action_Description'].tolist()
quarters = all_pbp['Period'].tolist()
team_action = all_pbp['Team'].tolist()


N = len(team_names)
T = len(all_pbp)
print(N,T)

solution_X_ij_B = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_B, quarters, team_action, away_team, total_quarters)
if solution_X_ij_B is not None:
    print("Solution for X_ij_B:")
    for i,row in enumerate(solution_X_ij_B):
        all_pbp.loc[:, f"X_{boxscore_B_players['#'].tolist()[i]}_j_B"] = row
        print(row)
else:
    print("No solution found.")

10 623
CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 1
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.033042
usertime: 0.033042
deterministic_time: 0.00153827
gap_integral: 0
solution_fingerprint: 0xb9dbbdfe701741cd

Solution for X_ij_B:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [59]:
all_pbp[380:395]

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,...,X_0_j_B,X_1_j_B,X_2_j_B,X_3_j_B,X_7_j_B,X_9_j_B,X_13_j_B,X_15_j_B,X_23_j_B,X_24_j_B
380,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:56,2,Shai Gilgeous-Alexander,offensive rebound,956.0,...,1,1,1,0,0,1,1,0,0,0
381,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,9,RJ Barrett,foul drawn,952.0,...,1,1,1,0,0,1,1,0,0,0
382,USA,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,9,Bobby Portis Jr,personal foul; 2 free throws awarded,952.0,...,1,1,1,0,0,1,1,0,0,0
383,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,13,Kelly Olynyk,Substitution out,952.0,...,1,1,1,0,0,1,1,0,0,0
384,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,2,Shai Gilgeous-Alexander,Substitution out,952.0,...,1,1,1,0,0,1,1,0,0,0
385,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,3,Melvin Ejim,Substitution in,952.0,...,1,1,1,1,1,0,0,0,0,0
386,CAN,https://www.fiba.basketball/api/img/team/logof...,80,89,Q3,0:52,24,Dillon Brooks,Substitution in,952.0,...,0,1,1,1,0,1,0,0,0,1
387,CAN,https://www.fiba.basketball/api/img/team/logof...,80,90,Q3,0:52,9,RJ Barrett,1st of 2 free throw made,952.0,...,0,1,1,1,0,1,0,0,0,1
388,CAN,https://www.fiba.basketball/api/img/team/logof...,80,90,Q3,0:52,2,Shai Gilgeous-Alexander,made the assist,952.0,...,0,1,1,1,0,1,0,0,0,1
389,CAN,https://www.fiba.basketball/api/img/team/logof...,80,91,Q3,0:52,9,RJ Barrett,2nd of 2 free throws made,952.0,...,0,1,1,1,0,1,0,0,0,1


In [62]:
i =0
sum_pred = 0
sum_actual = 0
for col in all_pbp.columns:
    if 'j_B' in col:
        print(f'For player number {col.split("_")[1]}: ----- We predict {sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))} vs {boxscore_B.time_seconds.iloc[i]} real seconds')
        sum_pred += sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))
        sum_actual += boxscore_B.time_seconds.iloc[i]
        i += 1

        print('-----------------------')
print('Total sum seconds predicted', sum_pred)
print('Total sum of real seconds', sum_actual)

For player number 0: ----- We predict 1930 vs 1930.0 real seconds
-----------------------
For player number 1: ----- We predict 1072 vs 1072.0 real seconds
-----------------------
For player number 2: ----- We predict 2413 vs 2413.0 real seconds
-----------------------
For player number 3: ----- We predict 433 vs 433.0 real seconds
-----------------------
For player number 7: ----- We predict 1826 vs 1826.0 real seconds
-----------------------
For player number 9: ----- We predict 2319 vs 2319.0 real seconds
-----------------------
For player number 13: ----- We predict 682 vs 682.0 real seconds
-----------------------
For player number 15: ----- We predict 138 vs 138.0 real seconds
-----------------------
For player number 23: ----- We predict 153 vs 153.0 real seconds
-----------------------
For player number 24: ----- We predict 2534 vs 2534.0 real seconds
-----------------------
Total sum seconds predicted 13500
Total sum of real seconds 13500.0


In [ ]:
all_pbp[(all_pbp.quarter=='4')][20:35]

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed,X_0_j_A,X_1_j_A,X_2_j_A,X_3_j_A,X_7_j_A,X_9_j_A,X_13_j_A,X_23_j_A,X_24_j_A,X_2_j_B,X_6_j_B,X_8_j_B,X_9_j_B,X_10_j_B,X_11_j_B,X_14_j_B,X_32_j_B,X_50_j_B,X_99_j_B
344,BRA,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,11,Gui Santos,foul drawn,420.0,4,24.0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1
345,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,23,Phil Scrubb,personal foul; 2 free throws awarded,420.0,4,0.0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1
346,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,3,Melvin Ejim,Substitution out,420.0,4,0.0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
347,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,13,Kelly Olynyk,Substitution out,420.0,4,0.0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
348,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,23,Phil Scrubb,Substitution out,420.0,4,0.0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1
349,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,24,Dillon Brooks,Substitution in,420.0,4,0.0,1,1,0,0,0,0,1,1,1,1,0,1,0,1,1,0,0,0,1
350,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,2,Shai Gilgeous-Alexander,Substitution in,420.0,4,0.0,1,1,1,0,0,0,0,1,1,1,0,1,0,1,1,0,0,0,1
351,CAN,https://www.fiba.basketball/api/img/team/logof...,52,51,Q4,7:00,7,Dwight Powell,Substitution in,420.0,4,0.0,1,1,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1
352,BRA,https://www.fiba.basketball/api/img/team/logof...,52,52,Q4,7:00,11,Gui Santos,1st of 2 free throws made,420.0,4,0.0,1,1,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1
353,BRA,https://www.fiba.basketball/api/img/team/logof...,52,52,Q4,7:00,2,Yago Santos,made the assist,420.0,4,0.0,1,1,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1


In [ ]:
all_pbp.to_csv('can_vs_bra.csv')